In [1]:
import numpy as np
import random
import os
import math
from sklearn import preprocessing
from sklearn.preprocessing import MinMaxScaler


from glob import glob
import pandas as pd
import cv2
from tqdm.auto import tqdm

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset

from torch.autograd import Variable

import torchvision.models as models
from torchvision import transforms

In [2]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"Using {device} device")

Using cuda device


In [3]:
torch.cuda.is_available()

True

In [4]:
torch.cuda.empty_cache()

### data

In [5]:
def get_train_data(data_dir):
    img_path_list = []
    label_list = []
    
    image_path = os.path.join(data_dir, 'image')
    label_path = os.path.join(data_dir, 'label')
    
    for product_name in os.listdir(image_path):
        product_path = os.path.join(image_path, product_name)
        if os.path.isdir(product_path):
            # get image path
            img_path_list.extend(glob(os.path.join(product_path, '*.jpg')))
            img_path_list.extend(glob(os.path.join(product_path, '*.png')))
            label = list(product_name[:5])
            
            # get label
            label_list.append(''.join(label))
                
    return img_path_list, label_list

In [6]:
img_list, label_list = get_train_data('./Data/product_image/Training/')

In [7]:
def data_blanced(img, label):
    x = []
    y = []
    
    for i in range(len(label)):
        _img = img[(i * 114): ((i + 1) * 114)]
        _label = label[i]
        
        for img_product in _img:
            x.append(img_product)
            y.append(_label)
            
    return x, y

In [8]:
x, y = data_blanced(img_list, label_list)

In [9]:
le = preprocessing.LabelEncoder()
targets = le.fit_transform(y)
targets = torch.as_tensor(targets)

In [10]:
len(targets)

8664

In [11]:
one_hot_y = F.one_hot(targets)

In [12]:
class CustomDataset(Dataset):
    def __init__(self, img_path_list, label_list, train_mode=True, transforms=None):
        self.transforms = transforms
        self.train_mode = train_mode
        self.img_path_list = img_path_list
        self.label_list = label_list

    def __getitem__(self, index):
        img_path = self.img_path_list[index]
        # Get image data
        image = cv2.imread(img_path)
        if self.transforms is not None:
            image = self.transforms(image)

        if self.train_mode:
            label = self.label_list[index]
            return image, label
        else:
            return image
    
    def __len__(self):
        return len(self.img_path_list)

In [13]:
train_transform = transforms.Compose([
                    transforms.ToTensor(),
                    transforms.Resize((256, 256)),
                    transforms.Normalize(mean=(0.5, 0.5, 0.5), std=(0.5, 0.5, 0.5))
                    ])

test_transform = transforms.Compose([
                    transforms.ToTensor(),
                    transforms.Resize((256, 256)),
                    transforms.Normalize(mean=(0.5, 0.5, 0.5), std=(0.5, 0.5, 0.5))
                    ])

In [14]:
train_len = int(len(x)*0.8)

train_img_path = x[:train_len]
train_label = one_hot_y[:train_len]

vali_img_path = x[train_len:]
vali_label = one_hot_y[train_len:]

In [15]:
train_dataset = CustomDataset(train_img_path, train_label, train_mode=True, transforms=train_transform)
train_loader = DataLoader(train_dataset, batch_size = 32, shuffle=True, num_workers=0, collate_fn=None)

vali_dataset = CustomDataset(vali_img_path, vali_label, train_mode=True, transforms=test_transform)
vali_loader = DataLoader(vali_dataset, batch_size = 5, shuffle=False, num_workers=0, collate_fn=None)

### model

In [16]:
class ResNet50(torch.nn.Module):
    def __init__(self):
        super(ResNet50, self).__init__()
        model = models.resnet50(pretrained=True)
        modules = list(model.children())[:-1]
        self.feature_extract = nn.Sequential(*modules)
        self.regressor = nn.Linear(2048, 76)
        

    def forward(self, x):
        x = self.feature_extract(x)
        # x = x.mean(dim=(-2, -1))
        # (batch, 2048, 4, 4)
        x = torch.flatten(x, start_dim=1)
        out = self.regressor(x)
        return out

### training

In [17]:
def validation(model, vali_loader, criterion, device):
    model.eval() # Evaluation
    vali_loss = []
    # regressor = nn.Linear(1000, 1).to(device)

    with torch.no_grad():
        for img, label in tqdm(iter(vali_loader)):
            img, label = img.float().to(device), label.float().to(device)

            logit = model(img)
            # logit = regressor(logit)
            logit = torch.squeeze(logit)
            loss = criterion(logit, label)
            
            vali_loss.append(loss.item())

    vali_mae_loss = np.mean(vali_loss)
    return vali_mae_loss

In [20]:
def train(model, optimizer, train_loader, vali_loader, scheduler, device):
    
    # model = ViT('L_32_imagenet1k', pretrained=True).to(device)
    # modules = list(model.children())[:-1]
    # ViT_seq = nn.Sequential(*modules)
    # regressor = nn.Linear(1000, 1).to(device)
    model.to(device)
    
    epochs = 20

    loss_plot = []
    vali_mae_plot = []

    # Loss Function
    criterion = torch.nn.CrossEntropyLoss()
    best_mae = 9999
    
    for epoch in range(1,epochs+1):
        model.train()
        train_loss = []
        for img, label in tqdm(iter(train_loader)):
            img, label = img.float().to(device), label.float().to(device)
            
            optimizer.zero_grad()

            # Data -> Model -> Output
            # logit = ViT_seq(img)
            logit = model(img)
            print(logit.shape)
            # logit = regressor(logit)
            logit = torch.squeeze(logit)
            # Calc loss
            loss = criterion(logit, label)
            loss_plot.append(loss)

            # backpropagation
            loss.backward()
            optimizer.step()

            train_loss.append(loss.item())
            
        if scheduler is not None:
            scheduler.step()
            
        # Evaluation Validation set
        vali_mae = validation(model, vali_loader, criterion, device)
        vali_mae_plot.append(vali_mae)
        
        print(f'Epoch [{epoch}] Train loss : [{np.mean(train_loss):.5f}] Validation loss : [{vali_mae:.5f}]\n')
        
        # Model Saved
        if best_mae > vali_mae:
            best_mae = vali_mae
            torch.save(model.state_dict(), './saved_models/resnet50.pth')
            print('Model Saved.')

In [ ]:
model = ResNet50()

optimizer = torch.optim.Adam(model.parameters(), lr=0.003)
scheduler = None

train(model, optimizer, train_loader, vali_loader, scheduler, device)

  0%|          | 0/217 [00:00<?, ?it/s]

torch.Size([32, 76])
torch.Size([32, 76])
torch.Size([32, 76])
torch.Size([32, 76])
torch.Size([32, 76])
torch.Size([32, 76])
torch.Size([32, 76])
torch.Size([32, 76])
torch.Size([32, 76])
torch.Size([32, 76])
torch.Size([32, 76])
torch.Size([32, 76])
torch.Size([32, 76])
torch.Size([32, 76])
torch.Size([32, 76])
torch.Size([32, 76])
torch.Size([32, 76])
torch.Size([32, 76])
torch.Size([32, 76])
torch.Size([32, 76])
torch.Size([32, 76])
torch.Size([32, 76])
torch.Size([32, 76])
torch.Size([32, 76])
torch.Size([32, 76])
torch.Size([32, 76])
torch.Size([32, 76])
torch.Size([32, 76])
torch.Size([32, 76])
torch.Size([32, 76])
torch.Size([32, 76])
torch.Size([32, 76])
torch.Size([32, 76])
torch.Size([32, 76])
torch.Size([32, 76])
torch.Size([32, 76])
torch.Size([32, 76])
torch.Size([32, 76])
torch.Size([32, 76])
torch.Size([32, 76])
torch.Size([32, 76])
torch.Size([32, 76])
torch.Size([32, 76])
torch.Size([32, 76])
torch.Size([32, 76])
torch.Size([32, 76])
torch.Size([32, 76])
torch.Size([3